In [ ]:
import torch
import torch.nn as nn
import numpy as np
from rightAlignedConv1d import rac1d
from tqdm import tqdm

In [ ]:
# with open("yuge.txt", "rt") as f:
with open("out", "rt") as f:
    # tdata = f.read().replace("\n","").replace("\t", "")
    tdata = f.read().replace("\t", "")

In [ ]:
unique_tokens = list(set(tdata))
token_to_idx = dict(zip(unique_tokens, range(len(unique_tokens))))
idx_to_token = dict(zip(range(len(unique_tokens)), unique_tokens))
idx_data = list(map(token_to_idx.get, tdata))

In [ ]:
def get_random_range(lenx, seq_len):
    start_i = np.random.randint(0, lenx-seq_len-1) # allow for one predicted and input token at the end
    return start_i, start_i + seq_len

In [ ]:
def generate_random_sample(data, seq_len):
    start_bound, end_bound = get_random_range(len(data), seq_len)
    return torch.tensor([data[start_bound: end_bound]], requires_grad=False), torch.tensor([data[start_bound+1: end_bound+1]], requires_grad=False)

In [ ]:
def generate_random_batch(data, seq_len, num_sample=100):
    xs, ys = [], []
    for _ in range(num_sample):
        gx, gy = generate_random_sample(data, seq_len)
        xs.append(gx)
        ys.append(gy)
    return torch.vstack(xs), torch.vstack(ys)

In [ ]:
def idxs_to_str(idxs):
    return "".join([idx_to_token[idx] for idx in idxs]).replace("\n"," <ENTER> ")

In [ ]:
def text_to_tens(text):
    return torch.tensor([list(map(token_to_idx.get, text))])

In [ ]:
def gen_one(text, model, sampler = lambda x: x.argmax()):
    new_char = sampler(model(text_to_tens(text).cuda())[0,:,-1]).item()
    return text + idx_to_token[new_char]

In [ ]:
def elaborate(text, model, num=10, sampler = lambda x: x.argmax()):
    ctex = text
    for _ in tqdm(range(num)):
        ctex = gen_one(ctex, model, sampler)
    return ctex

In [ ]:
def skippableMax(x: torch.Tensor) -> torch.Tensor:
    x_orig = x
    x = torch.cummax(x, 2).values
    return x_orig + x
    # return x

In [ ]:
class modelBlock(nn.Module):
    def __init__(self, nchannel, kernelsize) -> None:
        super().__init__()
        self.conv1 = rac1d(nchannel, nchannel, kernelsize, padding="same")
        self.conv2 = rac1d(nchannel, nchannel, kernelsize, padding="same")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x_orig = x
        x = self.conv1(x)
        x = nn.functional.gelu(x)
        # x = torch.relu(x)
        # x = skippableMax(x)

        x = self.conv2(x)
        x = nn.functional.gelu(x)
        # x = torch.relu(x)
        # x = skippableMax(x)

        output = x + x_orig # should already be relu-ed
        # output = torch.renorm(output, 2, -1, 1)
        return output

In [ ]:
class TModel(nn.Module):
    def __init__(self, char_num=67, kern_size=32, chan_count=140) -> None:
        super().__init__()

        self.embedding = nn.Embedding(char_num, chan_count)
        self.drop = nn.Dropout1d(p=0.1)
        self.block1 = modelBlock(chan_count, kern_size)

        self.block2 = modelBlock(chan_count, kern_size)
        self.block2_1 = modelBlock(chan_count, kern_size)
        self.block2_2 = modelBlock(chan_count, kern_size)

        self.block3 = modelBlock(chan_count, kern_size*2)
        self.output_conv = rac1d(chan_count, char_num, kern_size, padding="same")

    def forward(self, x: torch.Tensor, debug=False) -> torch.Tensor:
        # x = self.embedding(x).permute(1, 2, 0)
        x = self.embedding(x).permute(0, 2, 1)
        # if debug: print(f"{x=}")
        # if debug: print(f"{x.shape=}")
        x = self.drop(x)

        # x = self.block1(x)

        # x = self.block2(x)
        # x = self.block2_1(x)
        # x = self.block2_2(x)
        
        x = self.block3(x)
        
        # x = skippableMax(x)
        # x = skippableMax(x)
        
        x = self.output_conv(x)

        # if debug: print(f"{x=}")
        # if debug: print(f"{x.shape=}")
        return x

In [ ]:
# SEQ_LEN = 96
# SEQ_LEN = 256
SEQ_LEN = 128
BATCH_SIZE = 64
# BATCH_SIZE = 32
CHAR_COUNT = len(unique_tokens)

In [ ]:
model = TModel(CHAR_COUNT)
model = model.cuda()

In [ ]:
# model(generate_random_batch(idx_data, SEQ_LEN, BATCH_SIZE)[0].cuda(), True)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), capturable = True)

In [ ]:
model.train()
print(f"{model.training=}")
print(f"{model=}")

In [ ]:
# static_input = torch.randint(0, CHAR_COUNT, (BATCH_SIZE, SEQ_LEN), device='cuda').long()
# static_target = torch.randint(0, CHAR_COUNT, (BATCH_SIZE, SEQ_LEN), device='cuda').long()

In [ ]:
# s = torch.cuda.Stream()
# s.wait_stream(torch.cuda.current_stream())
# with torch.cuda.stream(s):
#     for i in range(3):
#         optimizer.zero_grad(set_to_none=True)
#         y_pred = model(static_input)
#         loss = criterion(y_pred, static_target)
#         loss.backward()
#         optimizer.step()
# torch.cuda.current_stream().wait_stream(s)

# # capture
# g = torch.cuda.CUDAGraph()
# # Sets grads to None before capture, so backward() will create
# # .grad attributes with allocations from the graph's private pool
# optimizer.zero_grad(set_to_none=True)
# with torch.cuda.graph(g):
#     static_y_pred = model(static_input)
#     static_loss = criterion(static_y_pred, static_target)
#     static_loss.backward()
#     optimizer.step()


In [ ]:
model.eval()
print(f"{model.training=}")
print(f"{model=}")

In [ ]:
# for epoch in tqdm(range(100)):
#     data, target = generate_random_batch(idx_data, SEQ_LEN, BATCH_SIZE)
#     static_input.copy_(data)
#     static_target.copy_(target)
#     g.replay()
#     if epoch % 100 == 0:
#         print(f"ep {epoch}")

#         outputs = model(data.cuda())

#         print(f"acc={torch.mean((outputs.argmax(1) == target.cuda()).float())}")
#         print(f"LAST SAMPLE EXP: {idxs_to_str(target[-1].tolist())}")
#         print(f"LAST SAMPLE MES: {idxs_to_str(outputs[-1].argmax(0).tolist())}")

#         print(f"{loss.item()=}")

In [ ]:
# for epoch in tqdm(range(100)):
#     data, target = generate_random_batch(idx_data, SEQ_LEN, BATCH_SIZE)
#     data = data.cuda()
#     target = target.cuda()
    
#     optimizer.zero_grad(set_to_none=True)
#     y_pred = model(data)
#     loss = criterion(y_pred, target)
#     loss.backward()
#     optimizer.step()

In [ ]:
torch.backends.cudnn.benchmark = True

In [ ]:
scripted_model = torch.jit.script(model)

In [ ]:
from LRTracker import LinearLRTracker
llrt = LinearLRTracker(5e-4, .0015, 10, 100)
# llrt = LinearLRTracker(5e-4, .003, 100, 100)

try:
    # Creates once at the beginning of training
    scaler = torch.cuda.amp.GradScaler()

    for epoch in tqdm(range(llrt.num_steps)):

        newLR = llrt.lr_step()
        # print(f"{newLR=}")
        for g in optimizer.param_groups:
            g['lr'] = newLR
        
        optimizer.zero_grad(set_to_none=True)
        data, target = generate_random_batch(idx_data, SEQ_LEN, BATCH_SIZE)
        data = data.cuda()
        target = target.cuda()

        # Casts operations to mixed precision
        with torch.cuda.amp.autocast():
            y_pred = scripted_model(data)
            loss = criterion(y_pred, target)

        # Scales the loss, and calls backward()
        # to create scaled gradients
        scaler.scale(loss).backward()

        # Unscales gradients and calls
        # or skips optimizer.step()
        scaler.step(optimizer)

        # Updates the scale for next iteration
        scaler.update()
        llrt.record_loss(loss.item())
except StopIteration as ex:
    print(f"Stopped because: {ex}")

EXLR = llrt.findLR()
print(f"{EXLR=}")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(llrt.steps[:len(llrt.losses)], llrt.losses)

In [ ]:
model = TModel(CHAR_COUNT)
model = model.cuda()
optimizer = optim.Adam(model.parameters(), capturable = True, lr=EXLR)
scripted_model = torch.jit.script(model)

In [ ]:
# list(model.parameters())

In [30]:
# Creates once at the beginning of training
scaler = torch.cuda.amp.GradScaler()

for epoch in tqdm(range(50000)):
   # optimizer.zero_grad()
   optimizer.zero_grad(set_to_none=True)
   data, target = generate_random_batch(idx_data, SEQ_LEN, BATCH_SIZE)
   data = data.cuda()
   target = target.cuda()
   
   # Casts operations to mixed precision
   with torch.cuda.amp.autocast():
      y_pred = scripted_model(data)
      loss = criterion(y_pred, target)

   # Scales the loss, and calls backward()
   # to create scaled gradients
   scaler.scale(loss).backward()

   # Unscales gradients and calls
   # or skips optimizer.step()
   scaler.step(optimizer)

   # Updates the scale for next iteration
   scaler.update()

   if epoch % 100 == 0:
      with torch.no_grad():
         print(f"ep {epoch}")
         outputs = scripted_model(data.cuda())
         print(f"acc={torch.mean((outputs.argmax(1) == target.cuda()).float())}")
         print(f"LAST SAMPLE EXP: {idxs_to_str(target[-1].tolist())}")
         print(f"LAST SAMPLE MES: {idxs_to_str(outputs[-1].argmax(0).tolist())}")
         print(f"{loss.item()=}")

 29%|██▉       | 14712/50000 [04:10<09:41, 60.71it/s]

ep 14700
acc=0.837646484375
LAST SAMPLE EXP:    super.onCreate(savedInstanceState); <ENTER>         // Sets the content view to the xml file. <ENTER>         setContentView(R.layout.main_ac
LAST SAMPLE MES:     uper(onCreate()acedInstanceState); <ENTER>         // Set  the content view to the rme tile  <ENTER>          etContentValw(r.layout.main_ac
loss.item()=0.6408604979515076


 30%|██▉       | 14808/50000 [04:12<10:33, 55.58it/s]

ep 14800
acc=0.8421630859375
LAST SAMPLE EXP:  <ENTER>                  * Hook method that decides whether to terminate the <ENTER>                  * phaser or not. <ENTER>                  */ <ENTER>     
LAST SAMPLE MES:       *             cook method that rerises whenher th shr inato aoe                  * Ehrled ou doo . *       *      e / <ENTER>     
loss.item()=0.6221038103103638


 30%|██▉       | 14912/50000 [04:14<09:34, 61.04it/s]

...

 35%|███▍      | 17307/50000 [04:54<08:52, 61.39it/s]

ep 17300
acc=0.8154296875
LAST SAMPLE EXP: makeStrategy  <ENTER>                                  (List<Resource> resources,  <ENTER>                                   LeasePool.SyncStrat
LAST SAMPLE MES:  ete taitegy =                                  Sist<Sesource> resources, <ENTER>  <ENTER>                                   LiasePool.SyncStrat
loss.item()=0.7232285737991333


 35%|███▍      | 17411/50000 [04:55<09:10, 59.22it/s]

ep 17400
acc=0.8565673828125
LAST SAMPLE EXP:     /** <ENTER>      * Constructor creates a new interpreter that handles user input <ENTER>      * supplied via the given {@code format} using 
LAST SAMPLE MES:      ** <ENTER>      * Ronstructor ioeates alLew Lnterpreter that handles ased input      * sumplied vie the given {@code nirmat} csing 
loss.item()=0.5548886060714722


 35%|███▍      | 17488/50000 [04:57<09:32, 56.77it/s]

In [ ]:
print(elaborate("for(", model, 150))